# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import re
import os
import configparser
import boto3
from io import StringIO
from pyspark.sql import SparkSession

config = configparser.ConfigParser()
config.read('./dwh.cfg')

AWS_ACCESS_KEY = config.get('AWS', 'AWS_KEY_ID')
AWS_SECRET = config.get('AWS','AWS_SECRET')
AWS_REGION = config.get('AWS','REGION')
S3_BUCKET = config.get('S3','BUCKET_NAME')

os.environ["AWS_ACCESS_KEY_ID"]= AWS_ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"]= AWS_SECRET

spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11,org.apache.hadoop:hadoop-aws:2.7.2").enableHiveSupport().getOrCreate()

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [14]:
from pyspark.sql.functions import from_unixtime, unix_timestamp, to_date, expr,\
                                  date_add,udf,col,avg,mean,year,month,split
from pyspark.sql.types import StringType, DateType

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

#### Exploring the I94 Data

In [15]:
# Read in the data here
df_i94_sample = pd.read_csv('immigration_data_sample.csv')

In [16]:
df_i94_sample.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


**Observations:** 
- The columns in the immigration data are encoded. Therefore these columns could be Foreign Keys in the Fact table
- To create the corresponding Dimension Tables the `SAS_Label_Descriptions` file will be used to extract them into `csv` files which will be stored in `meta_data` folder

#### Extracting meta data from i94 immigration data label Descriptions

***1.1 Extracting Raw Data***

In [17]:
# Reading the Label Description file and extracting information which can form dimensional tables later
with open('I94_SAS_Labels_Descriptions.SAS') as i94_descritpion:
    i94_label_content = i94_descritpion.read()

In [18]:
text = re.search(r'value i94cntyl.*? ;', i94_label_content, re.DOTALL).group()
text=text.split('\n')

In [19]:
i94cit_i94res = pd.DataFrame(columns=['i94_code','country_name'])

In [20]:
i94cit_i94res['i94_code'] = [code.split('=')[0].strip() for code in text[1:]]
i94cit_i94res['country_name'] = [code.split('=')[1].rstrip(' ;').replace("'","") for code in text[1:]]

In [21]:
i94cit_i94res.head(10)

,i94_code,country_name
0,582,"MEXICO Air Sea, and Not Reported (I-94, no l..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA
5,324,ANGOLA
6,529,ANGUILLA
7,518,ANTIGUA-BARBUDA
8,687,ARGENTINA
9,151,ARMENIA


**Observation:** This data can be used to map the country codes to their respective names

In [22]:
text = re.search(r'i94prtl.*?;', i94_label_content, re.DOTALL).group()
text=text.split('\n')

In [23]:
i94port_i94code = pd.DataFrame(columns=['i94_port_code','i94_airport_location','i94_airport_state'])

In [24]:
i94port_i94code['i94_port_code'] = [re.search('[A-Z](?=)\w+', code).group() for code in text[1:-1]]
i94port_i94code['i94_airport_location'] = [re.findall('\'(.*?),', code.split('\t=\t')[1])[0] if len(re.findall('\'(.*?),', code.split('\t=\t')[1]))>0 else code.split('\t=\t')[1].strip(" '") 
                                           for code in text[1:-1]]

In [25]:
i94port_i94code['i94_airport_state'] = [code.split('\t=\t')[1].strip(" '") for code in text[1:-1]]
i94port_i94code['i94_airport_state'] = i94port_i94code['i94_airport_state'].str.split(', ')
i94port_i94code['i94_airport_state'] = [code[-1] for code in i94port_i94code['i94_airport_state']]

In [26]:
for code in i94port_i94code.i94_airport_state:
    if len(code)>2 and 'Collapsed' not in code and 'No PORT Code' not in code:
        tmp = code.split(' ')
        if len(tmp)>1:
            if len(tmp[0])==2 and '(BPS)' in tmp[1] or '#ARPT' in tmp[1]:
                i94port_i94code.loc[i94port_i94code.i94_airport_state==code,'i94_airport_state'] = tmp[0]

            if len(tmp[1]) == 2 and len(tmp[0])!=2:
                i94port_i94code.loc[i94port_i94code.i94_airport_state==code,'i94_airport_state'] = tmp[1]

In [27]:
i94port_i94code.head(10)

,i94_port_code,i94_airport_location,i94_airport_state
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK
5,DTH,DUTCH HARBOR,AK
6,EGL,EAGLE,AK
7,FRB,FAIRBANKS,AK
8,HOM,HOMER,AK
9,HYD,HYDER,AK


**Observations:**
- This table can help in mapping the `i94port` from the immigration data and its `airport_location` and `state_code` in US

In [28]:
i94port_i94code.to_csv('i94_meta_data/i94port_i94code.csv',index=False)

In [29]:
text = re.search(r'value i94addrl.*? ;', i94_label_content, re.DOTALL).group()
text=text.split('\n')

In [30]:
i94_state_code = pd.DataFrame(columns=['i94_state_code','i94_state_name'])

In [31]:
i94_state_code['i94_state_code'] = [code.split("'='")[0].strip("\t'") for code in text[1:]]
i94_state_code['i94_state_name'] = [code.split("'='")[1].strip(" ';") for code in text[1:]]

In [32]:
i94_state_code.to_csv('i94_meta_data/i94_state_code.csv',index=False)

***1.2 Combining all the above steps***

In [33]:
data_dict = {}
key_name=''
for line in i94_label_content.split("\n"):
    line = re.sub(r"\s+", " ", line)
    if '/*' in line and '-' in line:
        line = line.strip('/*')
        key_name = line.split('-')[0].strip()
        data_dict[key_name] = []
    if '=' in line and key_name!='':
        data_dict[key_name].append([item.strip(';').strip(" ").replace('\'', '').lstrip().rstrip() for item in line.split('=')])

In [34]:
for key in data_dict:
    if len(data_dict[key])>0:
        if 'CIT' in key and 'RES' in key:
            i94cit_i94res = pd.DataFrame(data_dict[key],columns=['i94_country_code','country_name'])
            i94cit_i94res.loc[i94cit_i94res.country_name.str.contains('MEXICO'),'country_name'] = 'MEXICO'
            i94cit_i94res.to_csv('i94_meta_data/i94cit_i94res.csv',index=False)
        if 'PORT' in key:
            i94port_i94code = pd.DataFrame(data_dict[key],columns=['i94_port_code','i94_airport_location'])
            i94port_i94code[['port_city', 'port_state']] = i94port_i94code['i94_airport_location'].str.rsplit(',', 1, expand=True)
            i94port_i94code.drop(['i94_airport_location'], axis=1, inplace=True)
            i94port_i94code.to_csv('i94_meta_data/i94port_i94code.csv',index=False)
        if 'MODE' in key:
            i94mode = pd.DataFrame(data_dict[key],columns=['i94_mode_code','i94_mode'])
            i94mode.to_csv('i94_meta_data/i94mode.csv',index=False)
        if 'ADDR' in key:
            i94addr = pd.DataFrame(data_dict[key],columns=['i94_state_code','i94_state_name'])
            i94addr.to_csv('i94_meta_data/i94addr.csv',index=False)
        if 'VISA' in key:
            i94visa = pd.DataFrame(data_dict[key],columns=['i94_visa_code','i94_visa'])
            i94visa.to_csv('i94_meta_data/i94visa.csv',index=False)

In [35]:
i94cit_i94res.head()

,i94_country_code,country_name
0,582,MEXICO
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA


#### Exploring the SAS data

In [2]:
!ls ../../data/18-83510-I94-Data-2016/

i94_apr16_sub.sas7bdat	i94_jan16_sub.sas7bdat	i94_may16_sub.sas7bdat
i94_aug16_sub.sas7bdat	i94_jul16_sub.sas7bdat	i94_nov16_sub.sas7bdat
i94_dec16_sub.sas7bdat	i94_jun16_sub.sas7bdat	i94_oct16_sub.sas7bdat
i94_feb16_sub.sas7bdat	i94_mar16_sub.sas7bdat	i94_sep16_sub.sas7bdat


In [3]:
df_i94 =spark.read.format('com.github.saurfang.sas.spark').option("inferSchema", "true").\
                 option("dateFormat", "yyyyMMdd").\
                 load('../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat')

In [5]:
# Printing shape of the dataset
print("Number of Columns: {}".format(len(df_i94.columns)))
print("Number of Rows: {}".format(df_i94.count()))

In [46]:
df_i94.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [3]:
%%time
for file in os.listdir('../../data/18-83510-I94-Data-2016/'):
    path='../../data/18-83510-I94-Data-2016/{}'.format(file)
    df_i94 = spark.read.format('com.github.saurfang.sas.spark').option("inferSchema", "true").option("dateFormat", "yyyyMMdd").load(path)
    df_i94=df_i94.drop('validres','delete_days','delete_mexl','delete_dup','delete_recdup','delete_visa')
    bucket_path = r"s3a://{}/i94_raw_data/{}_sasdata.parquet".format(S3_BUCKET,file.strip('.sas7bdat').split('_')[1])
    print(bucket_path)
    df_i94.write.parquet(bucket_path, "overwrite")

s3a://dend-capstone-data/i94_raw_data/apr16_sasdata.parquet
s3a://dend-capstone-data/i94_raw_data/sep16_sasdata.parquet
s3a://dend-capstone-data/i94_raw_data/nov16_sasdata.parquet
s3a://dend-capstone-data/i94_raw_data/mar16_sasdata.parquet
s3a://dend-capstone-data/i94_raw_data/jun16_sasdata.parquet
s3a://dend-capstone-data/i94_raw_data/aug16_sasdata.parquet
s3a://dend-capstone-data/i94_raw_data/may16_sasdata.parquet
s3a://dend-capstone-data/i94_raw_data/jan16_sasdata.parquet
s3a://dend-capstone-data/i94_raw_data/oct16_sasdata.parquet
s3a://dend-capstone-data/i94_raw_data/jul16_sasdata.parquet
s3a://dend-capstone-data/i94_raw_data/feb16_sasdata.parquet
s3a://dend-capstone-data/i94_raw_data/dec16_sasdata.parquet
CPU times: user 323 ms, sys: 92.8 ms, total: 416 ms
Wall time: 39min 56s


In [47]:
%%time
for file in os.listdir('../../data/18-83510-I94-Data-2016/'):
    path='../../data/18-83510-I94-Data-2016/{}'.format(file)
    print(path)
    df_i94 = spark.read.format('com.github.saurfang.sas.spark').option("inferSchema", "true").option("dateFormat", "yyyyMMdd").load(path)
    df_i94=df_i94.drop('validres','delete_days','delete_mexl','delete_dup','delete_recdup','delete_visa')
    df_i94_total = df_i94.union(df_i94)

../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat
CPU times: user 41.2 ms, sys: 8.2 ms, total: 49.4 ms
Wall time: 1.27 s


In [48]:
# Printing shape of the combined dataset shape
print("Number of Columns: {}".format(len(df_i94_total.columns)))
print("Number of Rows: {}".format(df_i94_total.count()))

Number of Columns: 28
Number of Rows: 6865980


**Observations:** From the dataset and the SAS_Label description file:
- The column `cicid` is like uuid to me
- The columns `validres, delete_days, delete_mexl, delete_dup, delete_recdup, delete_visa` are additional columns and can be removed by inferring from the sample csv file

#### Exploring the Airport Codes Data

In [52]:
df_airport = spark.read.format('csv').options(header='true', inferSchema='true').load('airport-codes_csv.csv')

In [53]:
df_airport.limit(10).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"
5,00AS,small_airport,Fulton Airport,1100,NA,US,US-OK,Alex,00AS,None,00AS,"-97.8180194, 34.9428028"
6,00AZ,small_airport,Cordes Airport,3810,NA,US,US-AZ,Cordes,00AZ,None,00AZ,"-112.16500091552734, 34.305599212646484"
7,00CA,small_airport,Goldstone /Gts/ Airport,3038,NA,US,US-CA,Barstow,00CA,None,00CA,"-116.888000488, 35.350498199499995"
8,00CL,small_airport,Williams Ag Airport,87,NA,US,US-CA,Biggs,00CL,None,00CL,"-121.763427, 39.427188"
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350,NA,US,US-CA,Pine Valley,00CN,None,00CN,"-116.4597417, 32.7273736"


In [54]:
df_airport.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [55]:
# Correcting data types
df_airport=df_airport.withColumn("elevation_ft", df_airport["elevation_ft"].cast('float'))
df_airport=df_airport.drop_duplicates()
df_airport = df_airport.withColumn('iso_region', split(df_airport['iso_region'], '-').getItem(1))

In [56]:
df_airport = df_airport.withColumn('latitude_deg', split(df_airport['coordinates'], ',').getItem(0).cast('float'))
df_airport = df_airport.withColumn('longitude_deg', split(df_airport['coordinates'], ',').getItem(1).cast('float'))
df_airport=df_airport.drop('coordinates')

In [57]:
df_airport.limit(10).toPandas()

,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,latitude_deg,longitude_deg
0,small_airport,The Farm Airport,375.0,NA,US,GA,Wrightsville,01GE,None,01GE,-82.771103,32.674400
1,small_airport,Rustic Wilderness Airport,190.0,NA,US,AK,Willow,02AK,None,02AK,-150.098007,61.876900
2,heliport,West Virginia Univ. Hosp. Inc. Gnd. Pad #2 Hel...,1137.0,NA,US,WV,Morgantown,04WV,None,04WV,-79.956108,39.653610
3,heliport,St Joseph Hospital Heliport,470.0,NA,US,MD,Towson,08MD,None,08MD,-76.610001,39.388302
4,closed,Silver Cross Hospital Heliport,610.0,NA,US,IL,Joliet,None,None,None,-88.052277,41.536182
5,heliport,Christus Spohn Hospital Memorial Heliport,38.0,NA,US,TX,Corpus Christi,0TE1,None,0TE1,-97.414200,27.779800
6,seaplane_base,Loring Seaplane Base,0.0,NA,US,AK,Loring,13Z,WLR,13Z,-131.636993,55.601299
7,heliport,Switch Pad Heliport,496.0,NA,US,KY,Louisville,1KY5,None,1KY5,-85.701302,38.166698
8,closed,[Duplicate] Hoby Wolf Airport,600.0,NA,US,MD,Eldersburg,1W5,None,1W5,-76.927498,39.407902
9,heliport,Madigan Hospital Heliport,300.0,NA,US,WA,Fort Lewis,1WA8,None,1WA8,-122.550003,47.106499


In [58]:
df_airport.printSchema()

root
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: float (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- latitude_deg: float (nullable = true)
 |-- longitude_deg: float (nullable = true)



In [ ]:
# Printing Shape of the Dataset
print("Number of Columns: {}".format(len(df_airport.columns)))
print("Number of Rows: {}".format(df_airport.count()))

#### Exploring the US cities Demographics Data

In [59]:
df_demographics = spark.read.format('csv').options(header='true', inferSchema='true',delimiter=';').load('us-cities-demographics.csv')

In [60]:
df_demographics.limit(10).toPandas()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402
5,Peoria,Illinois,33.1,56229,62432,118661,6634,7517,2.40,IL,American Indian and Alaska Native,1343
6,Avondale,Arizona,29.1,38712,41971,80683,4815,8355,3.18,AZ,Black or African-American,11592
7,West Covina,California,39.8,51629,56860,108489,3800,37038,3.56,CA,Asian,32716
8,O'Fallon,Missouri,36.0,41762,43270,85032,5783,3269,2.77,MO,Hispanic or Latino,2583
9,High Point,North Carolina,35.5,51751,58077,109828,5204,16315,2.65,NC,Asian,11060


In [61]:
df_demographics.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)



### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

### move files to S3

In [7]:
import configparser
import boto3
import pandas as pd
from io import StringIO

config = configparser.ConfigParser()
config.read('./dwh.cfg')

AWS_ACCESS_KEY = config.get('AWS', 'AWS_KEY_ID')
AWS_SECRET = config.get('AWS','AWS_SECRET')
AWS_REGION = config.get('AWS','REGION')
S3_BUCKET = config.get('S3','BUCKET_NAME')

In [10]:
s3 = boto3.resource('s3',region_name=AWS_REGION,
                    aws_access_key_id=AWS_ACCESS_KEY,
                    aws_secret_access_key=AWS_SECRET)

In [11]:
csv_buffer = StringIO()
df_airports = pd.read_csv('airport-codes_csv.csv')
df_airports.to_csv(csv_buffer)
s3.Object(S3_BUCKET,'airport_codes/airport-codes_csv.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '19F04C1FF73127C2',
  'HostId': 'k3HL5Z2nAKNSyZzF0ZBo8r3FjGu25hPNDcYvJvoR76u3F+Anp/hMHlZq5t+TPiMYXBcrvno/zOk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'k3HL5Z2nAKNSyZzF0ZBo8r3FjGu25hPNDcYvJvoR76u3F+Anp/hMHlZq5t+TPiMYXBcrvno/zOk=',
   'x-amz-request-id': '19F04C1FF73127C2',
   'date': 'Mon, 11 May 2020 18:37:02 GMT',
   'etag': '"75285372d53b7508543b5c671383a2fe"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"75285372d53b7508543b5c671383a2fe"'}

In [12]:
df_demographics = pd.read_csv('us-cities-demographics.csv')
df_demographics.to_csv(csv_buffer)
s3.Object(S3_BUCKET,'us-demographics/us-cities-demographics.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'A7D85AA646A9DBCF',
  'HostId': 'twCyiYQxiy9gukylHvfVolx7NdbXHCWm66vW+KiyqMTcanwLyBsQDCHl+BBik3PElJ49rEeIcgw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'twCyiYQxiy9gukylHvfVolx7NdbXHCWm66vW+KiyqMTcanwLyBsQDCHl+BBik3PElJ49rEeIcgw=',
   'x-amz-request-id': 'A7D85AA646A9DBCF',
   'date': 'Mon, 11 May 2020 18:37:20 GMT',
   'etag': '"175d3f6963129a4bd9dec03c4caaa452"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"175d3f6963129a4bd9dec03c4caaa452"'}

In [ ]:
out_buffer = BytesIO()
df_i94.to_parquet(out_buffer, index=False)